# Mod 5 Project - Sandy's Notebook

## Imports and Data Ingestion

In [30]:
#import libraries

import mod_5_project_helper as hp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


#set environment - REMEMBER TO COPY OVER mod_5_project_helper.py

hp.set_environment()
%matplotlib inline
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

#initiliase variables

%run variables.py

#import data

df = pd.read_csv("data.csv")

## Data Cleanup

In [2]:
df.drop_duplicates(inplace = True) #drop 10,728 duplicate rows

In [3]:
df.drop(index = 100000, axis = 0, inplace = True) #delete last blank row in the data set

In [4]:
df.drop(df[df["Current Loan Amount"] == 99999999].index, inplace = True) #delete 11484 rows where the loan amount is 99999999

In [5]:
df.dropna(subset = ['Credit Score'], inplace = True) #delete 19154 rows where Credit Score is missing

In [6]:
df.dropna(subset = ['Years in current job'], inplace = True) #delete 2564 rows where Years in current job is missing 

In [7]:
df["Years in current job"].replace(to_replace = "year.*", value = "",inplace = True, regex = True) #remove 'years' or 'year' from string

In [8]:
df.drop(df[df["Home Ownership"] == "HaveMortgage"].index, inplace = True) #delete 120 rows with "HaveMortgage" as meaning isn't clear

In [9]:
df.dropna(subset = ['Months since last delinquent'], inplace = True) #delete 30,000 rows where data is missing 

In [10]:
df.drop(columns = "Maximum Open Credit", inplace = True) #delete Maximum Open Credit columns as some of these numbers are huge e.g. 798255370.0

In [11]:
df = hp.drop_column_keyword_search(df, ["Loan ID", "Customer ID"]) #delete features 'Loan ID' and 'Customer ID' as they don't add anything

In [12]:
df = df.fillna(df.median()) #fill remaining values with 

In [13]:
#rename and consolidate categorical variables for purpose

df["Purpose"].replace({
              "Business Loan": "business_loan",
              "Medical Bills": "medical bills",
              "Educational Expenses": "educational_expenses",
              "Buy House": "buy_house",
              "Buy a Car": "buy_a_car",
              "Debt Consolidation": "debt_consolidation",
              "Home Improvements": "home_improvements",
              "Take a Trip": "take_a_trip",
              "vacation": "take_a_trip",
              "Major Purchase": "other",
              "Other": "other", 
              "renewable_energy": "home_improvements",
              "small_business": "business_loan",
              "moving": "home_improvements",
              "major_purchase": "major_purchase",
              "wedding": "wedding"
              }, inplace=True)

In [14]:
#rename categorical variables for Home Ownership

df["Home Ownership"].replace({
              "Home Mortgage": "mortgage",
              "Rent": "rent",
              "Own Home": "own_home",
              }, inplace=True)

In [15]:
#rename categorical variables for Term

df["Term"].replace({
              "Long Term": "long_term",
              "Short Term": "short_term",
              }, inplace=True)

In [16]:
#rename categorical variables for Loan Status

df["Loan Status"].replace({
              "Fully Paid": "fully_paid",
              "Charged Off": "default",
              }, inplace=True)

In [17]:
#rename columns to make the dataset easier to work with using . notation

df.columns = ['loan_status',
              'loan_amount',
              'term',
              'credit_score',
              'annual_income',
              'years_in_current_job',
              'home_ownership',
              'loan_purpose',
              'monthly_debt',
              'years_of_credit_history',
              'months_since_last_delinquent',
              'number_of_open_accounts',
              'number_of_credit_problems',
              'current_credit_balance',
              'bankruptcies',
              'tax_liens']

In [18]:
df.reset_index(inplace = True); #reset index

## Cleaned Dataset Information

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100514 entries, 0 to 100513
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       100000 non-null  object 
 1   Customer ID                   100000 non-null  object 
 2   Loan Status                   100000 non-null  object 
 3   Current Loan Amount           100000 non-null  float64
 4   Term                          100000 non-null  object 
 5   Credit Score                  80846 non-null   float64
 6   Annual Income                 80846 non-null   float64
 7   Years in current job          95778 non-null   object 
 8   Home Ownership                100000 non-null  object 
 9   Purpose                       100000 non-null  object 
 10  Monthly Debt                  100000 non-null  float64
 11  Years of Credit History       100000 non-null  float64
 12  Months since last delinquent  46859 non-null

In [32]:
df.shape

(100514, 19)

In [33]:
df.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0


## Train, test split, sampling and K-folds

In [19]:
default_df = df[df.loan_status == "default"] #create a new dataframe of loan defaulters

In [20]:
fully_paid_df = df[df.loan_status == "fully_paid"] #create a new dataframe of fully paid loans

In [21]:
fully_paid_sample = fully_paid_df.sample(7479,random_state = 42) #sample the fully paid loans dataframe to a number equal to the default loan dataframe

In [22]:
update_df = pd.concat([default_df, fully_paid_sample]) #create a new dataframe with a 50/50 split of defaulters and paid loans

In [28]:
update_df.head()

,index,loan_status,loan_amount,term,credit_score,annual_income,years_in_current_job,home_ownership,loan_purpose,monthly_debt,years_of_credit_history,months_since_last_delinquent,number_of_open_accounts,number_of_credit_problems,current_credit_balance,bankruptcies,tax_liens
15060,52698,default,112310.0,short_term,745.0,1474324.0,6,own_home,debt_consolidation,16708.98,34.5,76.0,11.0,0.0,95798.0,0.0,0.0
11202,38586,fully_paid,167288.0,short_term,738.0,814720.0,10+,rent,debt_consolidation,10116.17,16.5,46.0,10.0,0.0,259559.0,0.0,0.0
24099,89512,default,562100.0,long_term,728.0,1553440.0,8,mortgage,debt_consolidation,27443.98,23.0,29.0,13.0,0.0,430825.0,0.0,0.0
9661,32858,fully_paid,229878.0,long_term,670.0,1132780.0,3,mortgage,debt_consolidation,17369.23,22.3,7.0,7.0,0.0,2916196.0,0.0,0.0
8446,28560,fully_paid,64944.0,short_term,704.0,1308853.0,1,mortgage,debt_consolidation,18214.92,11.6,13.0,9.0,0.0,93214.0,0.0,0.0


In [23]:
update_df = update_df.sample(frac = 1, random_state = 42) #shuffle the new 50/50 split dataframe

In [24]:
X = update_df.drop(columns = "loan_status") #create predictor dataframe

In [34]:
update_df.groupby(update_df.loan_status).count()

,index,loan_amount,term,credit_score,annual_income,years_in_current_job,home_ownership,loan_purpose,monthly_debt,years_of_credit_history,months_since_last_delinquent,number_of_open_accounts,number_of_credit_problems,current_credit_balance,bankruptcies,tax_liens
loan_status,,,,,,,,,,,,,,,,
default,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479
fully_paid,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479,7479


In [25]:
y = update_df.loan_status #create target variable dataframe

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #create train/test split

In [27]:
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=42) #initialise cross validation object

In [94]:
#export data for CSVs for ease

X_train.to_csv("X_train.csv")
X_test.to_csv("X_test.csv")
y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")

## Decision Trees

In [45]:
X_train.reset_index(inplace = True); #reset index

In [46]:
X_train.drop(columns = ["level_0", "index"], inplace = True); #drop old index columns

In [47]:
#make a dataframe of categorical variables

X_train_categorical = pd.concat([X_train.term, 
                                 X_train.years_in_current_job, 
                                 X_train.home_ownership, 
                                 X_train.loan_purpose], 
                                 axis = 1) 

In [48]:
#make a dataframe of continous variables

X_train_continuous = pd.concat([X_train.loan_amount, 
                                X_train.credit_score, 
                                X_train.annual_income, 
                                X_train.monthly_debt, 
                                X_train.years_of_credit_history,
                                X_train.months_since_last_delinquent,
                                X_train.number_of_open_accounts,
                                X_train.current_credit_balance,
                                X_train.bankruptcies,
                                X_train.tax_liens], 
                                axis = 1)

In [49]:
#one hot encode categorical variables
#for everything but decision trees we should do drop_first = True
X_train_one_hot_encoded = pd.get_dummies(X_train_categorical)

In [50]:
#stitch the one hot encoded dataframe back together

X_train_updated = pd.concat([X_train_continuous, X_train_one_hot_encoded], axis = 1)

In [51]:
#instantiate tree classifier object

tree_clf = DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 10, random_state = 42)

In [52]:
#fit model using cross validation. Need to check this to make sure the approach is accurate and use ROC curve
#with hyperparamter tuning

baseline_tree_model = cross_validate(tree_clf, 
                                X_train_updated, 
                                y_train, 
                                cv=crossvalidation, 
                                return_train_score = True)

baseline_tree_model

#tree_clf.fit(X_train_updated,y_train)

{'fit_time': array([0.11928415, 0.04472518, 0.0436852 , 0.04277205, 0.04314017,
        0.04189777, 0.04386377, 0.04632807, 0.04284215, 0.04228187]),
 'score_time': array([0.00759006, 0.00423598, 0.00507379, 0.00371194, 0.00366998,
        0.00349903, 0.00501394, 0.00538588, 0.00330281, 0.00344825]),
 'test_score': array([0.64828739, 0.66081871, 0.62238931, 0.65998329, 0.64076859,
        0.65079365, 0.65802676, 0.64214047, 0.63795987, 0.6312709 ]),
 'train_score': array([0.6557712 , 0.65196397, 0.65753552, 0.65270684, 0.65623549,
        0.6557712 , 0.65431755, 0.65673166, 0.65719591, 0.65524605])}

In [ ]:
#find evaluation metric for CLF object

In [55]:
#export tree to a dot file so it can be converted to an image using the CLI:
# dot -Tpng tree.dot -o tree.png

export_graphviz(tree_clf, 
                out_file = ("tree.dot"), 
                feature_names = X_train_updated.columns, 
                class_names = y_train.values, 
                rounded = True, 
                filled = True)

In [36]:
rnd_clf = RandomForestClassifier(n_estimators = 500,
                               max_leaf_nodes = 16,
                               n_jobs = -1,
                               random_state = 42)

In [123]:
baseline_forest_model = cross_validate(rnd_clf,
                                       X_train_updated,
                                       y_train,
                                       cv = crossvalidation,
                                       return_train_score = True)
baseline_forest_model

{'fit_time': array([2.62067509, 2.37249088, 2.73847508, 2.43558884, 2.38792706,
        2.4939611 , 2.34673882, 2.5359509 , 2.35448909, 2.36896491]),
 'score_time': array([0.21141291, 0.21368885, 0.21379304, 0.2152729 , 0.212672  ,
        0.21272492, 0.21909308, 0.21522403, 0.21227288, 0.21321702]),
 'test_score': array([0.65162907, 0.6566416 , 0.64076859, 0.65079365, 0.63659148,
        0.66499582, 0.65133779, 0.66973244, 0.66471572, 0.65217391]),
 'train_score': array([0.66700715, 0.66366422, 0.66635714, 0.6659857 , 0.66607856,
        0.6634785 , 0.66666667, 0.6632312 , 0.6643454 , 0.66573816])}

In [37]:
grid_params = [{'n_estimators' : [500, 600, 700, 800, 900, 1000],
                'max_features' : [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]}]

In [41]:
grid_search_random_forest = GridSearchCV(rnd_clf, grid_params, cv = crossvalidation, return_train_score = True)

In [53]:
grid_search_random_forest.fit(X_train_updated, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=16,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=500, n_jobs=-1,
                                              oob_score=F

In [54]:
grid_search_random_forest.best_params_

{'max_features': 18, 'n_estimators': 1000}

In [55]:
grid_search_random_forest.cv_results_

{'mean_fit_time': array([ 2.21225586,  2.0878819 ,  2.44974189,  2.80376911,  2.9700597 ,
         3.50117252,  2.17909062,  2.57878382,  2.89939718,  3.44554627,
         3.74630961,  4.11255682,  2.69546959,  2.79165008,  3.22223573,
         3.79034204,  4.00761888,  4.75887332,  2.75479052,  3.54295464,
         3.86126466,  4.28136194,  4.87407742,  5.43022313,  3.13828785,
         3.70023835,  4.30339413,  4.95367599,  5.48059928,  6.37810421,
         4.7456418 ,  5.84136319,  5.04358559,  5.59796865,  6.29653046,
         6.86260378,  3.94972184,  4.61423905,  5.48236017,  6.51252835,
         6.83500726,  7.85078938,  4.3424279 ,  5.1191236 ,  5.92152264,
         6.7282346 ,  7.52062311,  8.68328595,  4.91735568,  5.56901319,
         7.11079309,  7.33125362,  8.31801004,  9.32897203,  5.09386921,
         6.41328359,  7.10741868,  8.23026955,  9.16182854, 10.09367759]),
 'std_fit_time': array([0.99758259, 0.09517019, 0.1545531 , 0.16646824, 0.18970981,
        0.40384292, 0

In [59]:
best_forest_model_clf = RandomForestClassifier(n_estimators = 1000,
                                               max_leaf_nodes = 18,
                                               n_jobs = -1,
                                               random_state = 42)

In [60]:
random_forest_best_results = cross_validate(best_forest_model_clf, 
                                            X_train_updated, 
                                            y_train, 
                                            cv=crossvalidation, 
                                            return_train_score = True)

In [62]:
random_forest_best_results

{'fit_time': array([6.80627894, 9.42683792, 5.25640798, 5.53262615, 5.49912071,
        5.29805017, 5.42514992, 5.17917514, 4.87725997, 5.007797  ]),
 'score_time': array([0.52748418, 0.516747  , 0.52846909, 0.52315497, 0.42249823,
        0.41807485, 0.4221561 , 0.41293383, 0.41814995, 0.52153683]),
 'test_score': array([0.65079365, 0.66248956, 0.64578112, 0.65329992, 0.64160401,
        0.6641604 , 0.65133779, 0.67056856, 0.66638796, 0.65301003]),
 'train_score': array([0.67035008, 0.66654285, 0.66877147, 0.66914291, 0.66858576,
        0.66728573, 0.66898793, 0.66824513, 0.66852368, 0.66805942])}

In [ ]:
#do we need to maximize FPR or TPR for our business case?